### Connection

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from graphdatascience import GraphDataScience

URI = "neo4j://localhost:7687"
NAME_DB = "movies"

gds = GraphDataScience(URI, aura_ds=False, database=NAME_DB)

/home/angelo-dev/miniconda3/envs/data_science/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Graph Analytics

- similarità tra utente e tra film
- provare ad aggiungere proprietà di grado o degree
- trovare community
- confrontare community
- recommendation system in base a quale community ti trovi

In [ ]:
result = gds.run_cypher(
       """
          'movies-analyzed',  ['User', 'Movi  ['RATED', )   graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipProjection, relationshipCount AS rels
""      "
         )
res         ult                                                                                                               ,
               ,
                                                                                                      "\n",
                '"\n',
                '"\n',
                "\n",
            ,
         ,
                                                                                                         ,
         ,
                                                                                 '       """\n',
                        '""      "\n',
                                    "               ,\n",
                '                                                                                                      "\\n",\n',
                "                '\"\\n',\n",
                "                '\"\\n',\n",
                '                "\\n",\n',
                "            ,\n",
                "         ,\n",
                "                                                                                                         ,\n",
                "         ,\n",
                '                                                                                 \'       """\\n\',\n',
                '                        \'""      "\\n\',\n',
                "                                                '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                '                                                                                 \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                        \'    """\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                                                     '\\'\"n\\',\\n',\n",
                "                        '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                "                                                         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                       ,\n",
                "                                                         ,\n",
                "      ,\n",
                "                                       ,\n",
                "                                                                           ,\n",
                "      ,\n",
                "         ,",
         ,
                                                                                 '    """\n',
                                    '"""\n',
            ,
         ,
                                                                                       '    """\n',
                        '    """\n',
            ,
         ,
                                                                     '\'"n\',\n',
                        "                  ,       ,                 ,                                                                                               '\"\"n',\n",
                '                              ",\n',
                "                  ,\n",
                '                                                                                          \'    """n\',\n',
                "                                             '\"\"n',\n",
                "                     ,\n",
                "                     ,\n",
                '                                                                                                      \'    """n\',\n',
                '                                       \'    """n\',\n',
                "                           ,\n",
                "                           ,\n",
                "                                                                           '',\n",
                "                     ,\n",
                '                                                                                                   \'    """n\',\n',
                "                                             '\"\"n',\n",
                "                     ,\n",
                "                     ,\n",
                "                                                   ,\n",
                "                                                                     ,\n",
                "                  ,\n",
                "                                                   ,\n",
                '                "\\n",\n',
                "                ' ,\"\\n',\n",
                "                '    ,\"\\n',\n",
                '                "\\n",\n',
                "            ,\n",
                "         ,\n",
                "                                                                                                         ,\n",
                "         ,\n",
                '                                                                                 \'       """\\n\',\n',
                '                        \'""      "\\n\',\n',
                "                                                '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                '                                                                                 \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                        \'    """\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                                                     '\\'\"n\\',\\n',\n",
                "                        '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                "                                                         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                       ,\n",
                "                                                         ,\n",
                "      ,\n",
                "                                       ,\n",
                "                                                                           ,\n",
                "      ,\n",
                "         ,",
         ,
                                                         ,
                                                                                       '    """\n',
                                    '"""\n',
            ,
         ,
                                       ,
                                                         ,
      ,
                                       ,
                                                                           ,
      ,
         ,

In [ ]:
gds.run_cypher(
    """
    CALL gds.graph.list('movies-analyzed')
    YIELD density,degreeDistribution;
"""
)

## Similarity Analysis

In [ ]:
result = gds.run_cypher(
    """
    CALL gds.nodeSimilarity.stats('movies-analyzed') 
    """
)

In [ ]:
result["similarityDistribution"].to_dict()                                                                                                                                                '"n',
                   '"movies"n',
                  ,       ,                 ,                                                                                               '""n',
                              "result",
                  ,
                                                                                          '    """n',
                                             '""n',
                     ,
                     ,
                                                                                                      '    """n',
                                       '    """n',
                           ,
                           ,
                                                                           '',
                     ,
                                                                                                   '    """n',
                                             '""n',
                     ,
                     ,
                                                   ,
                                                                     ,
                  ,
                                                   ,
                "\n",
                ' ,"\n',
                '    ,"\n',
                "\n",
            ,
         ,
                                                                                                         ,
         ,
                                                                                 '       """\n',
                        '""      "\n',
                                    "               ,\n",
                '                                                                                                      "\\n",\n',
                "                '\"\\n',\n",
                "                '\"\\n',\n",
                '                "\\n",\n',
                "            ,\n",
                "         ,\n",
                "                                                                                                         ,\n",
                "         ,\n",
                '                                                                                 \'       """\\n\',\n',
                '                        \'""      "\\n\',\n',
                "                                                '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                '                                                                                 \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                        \'    """\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                                                     '\\'\"n\\',\\n',\n",
                "                        '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                "                                                         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                       ,\n",
                "                                                         ,\n",
                "      ,\n",
                "                                       ,\n",
                "                                                                           ,\n",
                "      ,\n",
                "         ,",
         ,
                                                                                 '    """\n',
                                    '"""\n',
            ,
         ,
                                                                                       '    """\n',
                        '    """\n',
            ,
         ,
                                                                     '\'"n\',\n',
                        "                  ,       ,                 ,                                                                                               '\"\"n',\n",
                '                              ",\n',
                "                  ,\n",
                '                                                                                          \'    """n\',\n',
                "                                             '\"\"n',\n",
                "                     ,\n",
                "                     ,\n",
                '                                                                                                      \'    """n\',\n',
                '                                       \'    """n\',\n',
                "                           ,\n",
                "                           ,\n",
                "                                                                           '',\n",
                "                     ,\n",
                '                                                                                                   \'    """n\',\n',
                "                                             '\"\"n',\n",
                "                     ,\n",
                "                     ,\n",
                "                                                   ,\n",
                "                                                                     ,\n",
                "                  ,\n",
                "                                                   ,\n",
                '                "\\n",\n',
                "                ' ,\"\\n',\n",
                "                '    ,\"\\n',\n",
                '                "\\n",\n',
                "            ,\n",
                "         ,\n",
                "                                                                                                         ,\n",
                "         ,\n",
                '                                                                                 \'       """\\n\',\n',
                '                        \'""      "\\n\',\n',
                "                                                '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                '                                                                                 \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                        \'    """\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                                                     '\\'\"n\\',\\n',\n",
                "                        '                \"n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                         ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                                                                                \'\\"\\"n\',\\n",\n',
                "                '                        \",\\n',\n",
                '                "         ,\\n",\n',
                "                '                                                                                 \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                "                '                        \\'    \"\"\"n\\',\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                \"                                                         '',\\n\",\n",
                '                "                                                         ,\\n",\n',
                "                '                                                                                       \\'    \"\"\"n\\',\\n',\n",
                '                "                                    \'\\"\\"n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                         ,\\n",\n',
                '                "      ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "         ,\n",
                "                                                         ,\n",
                '                                                                                       \'    """\\n\',\n',
                '                                    \'"""\\n\',\n',
                "            ,\n",
                "         ,\n",
                "                                       ,\n",
                "                                                         ,\n",
                "      ,\n",
                "                                       ,\n",
                "                                                                           ,\n",
                "      ,\n",
                "         ,",
         ,
                                                         ,
                                                                                       '    """\n',
                                    '"""\n',
            ,
         ,
                                       ,
                                                         ,
      ,
                                       ,
                                                                           ,
      ,
         ,

## Community Detection

In [ ]:
gds.run_cypher(
    """
    CALL gds.louvain.stats('movies-analyzed')
    YIELD communityCount, modularity, modularities
"""
)

## Recommendation Systems